Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель, способную классифицировать комментарии на позитивные и негативные. Для обучения предоставлен набор данных с разметкой о токсичности правок.

Необходимо построить модель со значением метрики качества *F1* не меньше 0.75. 


### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, hstack
# import matplotlib.pyplot as plt
import time
# from pymystem3 import Mystem
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression 
# from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
# import spacy
import lightgbm as lgbm
import torch
import transformers
from tqdm import notebook

/Users/sergeibovdei/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
nltk.download('punkt')
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sergeibovdei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sergeibovdei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sergeibovdei/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sergeibovdei/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
df_comments = pd.read_csv('/datasets/toxic_comments.csv')

In [4]:
print('Общее кол-во комментариев: ', len(df_comments))
print(f'Доля токсичных комментариев: {df_comments["toxic"].mean():6.2%}')

Общее кол-во комментариев:  159571
Доля токсичных комментариев: 10.17%


In [5]:
df_comments.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


#### Выводы после ознакомления с данными:
- Предоставлено почти 160 тысяч текстов на английском языке. Сразу видно, что каждый текст содержит не одно предложение, есть служебные символы (например, \n) и почти наверняка различные сокращения, жаргонизмы.
- Доля токсичных комментариев небольшая - всего 10%, что означает существенную несбалансированность нашего корпуса текстов.

### Подготовка лемм.

После ознакомления с возможностями библиотеки nltk представляется несколько возможных вариантов преобразования английского текста к набору лемм:

- На самом последнем этапе подготовки данных мы будем лемматизировать английские слова  при помощи Wordnet лемматизатора в nltk.
- Лемматизация может производиться с использованием разметки по частям речи (pos-tag), и это должно быть важным обстоятельством, поскольку без pos-tag лемматизатор nltk может и не преобразовать слово к лемме, если у разных частей речи разные леммы. Разметка pos_tag процесс длительный, тем не менее было принято решение проверить, как этап pos-тэгирования повлияет на точность модели. Соответственно, при обучении (и сравнении) моделей будут использоваться наборы лемм, полученных с pos-tag и без них.
- Наконец, очистка текста. На этапе очистки (а это первый этап предобработки) каждый текст и так разделяется на отдельные слова (токены), но это разделение не равносильно токенизатору на отдельные слова при помощи nltk. При обучении (и сравнении) моделей будут использоваться наборы лемм, в которых токенизация производилась при помощи nltk и при помощи обычного split по пробелам.
- Сформируем 2 набора лемм, варьируя обозначенные выше варианты предобработки. На предварительном этапе работы над проектом было сформировано 4 набора лемм и проведено исследование моделей линейной регрессии с каждым. Поскольку процесс лемматизации небыстрый, было принято решение оставить только два набора - достаточно для сравнения наборов лемм, но не требует такого кол-ва ресурсов:
  - nltk токенизация без очистки, затем лемматизация с pos-tag
  - очистка и токенизация просто по пробелам, затем лемматизация без pos-tag

In [6]:
def clear_text(text):
    return re.sub(r'[^a-zA-Z0-9 \']', ' ', text).split()

# данная функция необходима для лемматизации при преобразовании pos_tag nltk в pos_tag wordnet
def tag_wordnet(tag_nltk):
    if tag_nltk[0] == 'J':
        return wordnet.ADJ
    elif tag_nltk[0] == 'V':
        return wordnet.VERB
    elif tag_nltk[0] == 'R':
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [7]:
# в этом словаре мы сохраним полученные наборы лемм
lemmas_dict = {}
# и время для подготовки таких лемм
lemmas_time_dict = {}

In [8]:
%%time
"""Here we create lemmatized tokens using Wordnet tokenizer and pos-tag"""

dict_key = 'no clearing, tokenization, lemmatization WITH pos-tag'
start_time = time.time()
word_list = df_comments['text'].apply(nltk.word_tokenize)

# create Series of lists of tuples (token, nltk.pos_tag)
word_list_nltk_tagged = word_list.apply(nltk.pos_tag)

# mapping nltk pos_tag to wordnet.pos_tag
word_list_wordnet_tagged = word_list_nltk_tagged.apply(lambda x: [(w[0], tag_wordnet(w[1])) for w in x])

# create Series of lists of lemmas
lemmatizer = WordNetLemmatizer()
lemma_list = word_list_wordnet_tagged.apply(lambda x: [lemmatizer.lemmatize(w[0], w[1]) for w in x])

# create Series of strings of lemmas separated with space
lemmas_dict[dict_key] = lemma_list.apply(lambda x: ' '.join(x))
end_time = time.time()

lemmas_time_dict[dict_key] = round(end_time-start_time, 3)

CPU times: user 11min 39s, sys: 11.8 s, total: 11min 51s
Wall time: 12min 19s


In [9]:
%%time
"""Here we clear at first (and split to tokens) then lemmatize not using pos-tag"""

dict_key = "clearing, splitting, lemmatization WITHOUT pos-tag"
start_time = time.time()

# clear and split to tokens
word_list = df_comments['text'].apply(clear_text)

# create Series of lists of lemmas
lemmatizer = WordNetLemmatizer()
lemma_list = word_list.apply(lambda x: [lemmatizer.lemmatize(w) for w in x])

# create Series of strings of lemmas separated with space
lemmas_dict[dict_key] = lemma_list.apply(lambda x: ' '.join(x))
end_time = time.time()

lemmas_time_dict[dict_key] = round(end_time-start_time, 3)

CPU times: user 50.5 s, sys: 4.11 s, total: 54.6 s
Wall time: 57.2 s


In [10]:
# сравним результаты на каком-то примере
text_id = 0
print('raw text:')
print(df_comments.loc[text_id, 'text'])
for lemma in lemmas_dict:
    print(''.join(['_'] * 100))
    print(lemma, ':')
    print(lemmas_dict[lemma][text_id])

raw text:
Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
____________________________________________________________________________________________________
no clearing, tokenization, lemmatization WITH pos-tag :
Explanation Why the edits make under my username Hardcore Metallica Fan be revert ? They be n't vandalisms , just closure on some GAs after I vote at New York Dolls FAC . And please do n't remove the template from the talk page since I 'm retired now.89.205.38.27
____________________________________________________________________________________________________
clearing, splitting, lemmatization WITHOUT pos-tag :
Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They weren't vandalism just closure on some GAs after I voted at

Лемматизация без pos-tag и c pos-tag заметна невооруженным глазом (were - be). Проверим, как это скажется на моделях.

In [11]:
"""При построении модели с использованием BERT было замечено интересное свойство предоставленного корпуса текстов - 
если ограничивать длину текста, доля токсичных текстов увеличивается. Ниже продемонстрирована такая зависимость:"""

text_sizes = df_comments['text'].apply(len)

for length in [2500, 1000, 500, 200, 100, 50]:
    df_limited_length = df_comments[text_sizes < length]
    toxic_share = df_limited_length['toxic'].mean()
    print(f'Ограничение на длину текста: {length:4}, кол-во таких текстов: {len(df_limited_length): 10}, доля токсичных текстов: {toxic_share:.2%}')

Ограничение на длину текста: 2500, кол-во таких текстов:     156733, доля токсичных текстов: 10.16%
Ограничение на длину текста: 1000, кол-во таких текстов:     146187, доля токсичных текстов: 10.52%
Ограничение на длину текста:  500, кол-во таких текстов:     125544, доля токсичных текстов: 11.26%
Ограничение на длину текста:  200, кол-во таких текстов:      78186, доля токсичных текстов: 13.48%
Ограничение на длину текста:  100, кол-во таких текстов:      41575, доля токсичных текстов: 16.02%
Ограничение на длину текста:   50, кол-во таких текстов:      17324, доля токсичных текстов: 17.46%


Воспользуемся этим фактом при построении моделей - добавим в качестве признака длину текста в дополнение к вектору текста

# 2. Обучение с использованием векторизации tf-idf.

- Будет проведено исследование моделей для разных наборов лемм. 
- Векторизация лемм будет производиться методом TF-IDF. Это означает необходимость проведения векторизации уже после разделения выборки на обучающую и тестовую.
- Будет проведено сравнение моделей логистической регрессии и градиентного бустинга, с подбором гиперпараметров для достижения поставленной цели по метрике F1 = 0.75.
- Будет испробована техника upsampling для балансировки классов.

In [12]:
name_for = {
    'F1_test'     : 'F1 on test',
    'F1_valid'    : 'F1 on valid',
    'F1_train'    : 'F1 on train',
    'Accuracy'    : 'Accuracy on test',
    'time_fit'    : 'time to fit',
    'time_predict': 'time to predict',
    'time_lemma'  : 'time to vectorize',
    'ratio'       : 'possible overfitting'
}
all_results = pd.DataFrame(columns=[name_for[x] for x in name_for])
targets = df_comments['toxic']

In [13]:
RANDOM_STATE = 134
def fit_and_test(X_train, X_test, y_train, y_test, lemmas_key, feature_processing_describe, model_method, **kw_model):
    start_fit = time.time()
    model = model_method(**kw_model)
    model.fit(X_train, y_train)
    end_fit = time.time()
    y_test_predict = model.predict(X_test)
    y_train_predict = model.predict(X_train)
    end_predict = time.time()
    model_name = model.__class__.__name__ + ', ' + ', '.join([f'{i}={kw_model[i]}' for i in kw_model])
    index_row = model_name + '. Features preparing: ' + lemmas_key + feature_processing_describe
    f1_test = round(f1_score(y_test, y_test_predict), 3)
    f1_train = round(f1_score(y_train, y_train_predict), 3)
    all_results.loc[index_row, name_for['F1_test']] = f1_test
    all_results.loc[index_row, name_for['F1_train']] = f1_train
    all_results.loc[index_row, name_for['Accuracy']] = round(accuracy_score(y_test, y_test_predict), 3)
    all_results.loc[index_row, name_for['time_fit']] = round(end_fit - start_fit, 3)
    all_results.loc[index_row, name_for['time_predict']] = round(end_predict - end_fit, 3)
    all_results.loc[index_row, name_for['time_lemma']] = lemmas_time_dict[lemmas_key]
    all_results.loc[index_row, name_for['ratio']] = round(f1_train / f1_test,3)

def tfidf_features_split(lemmas_key, targets, test_size):
    lemmas_train, lemmas_test, y_train, y_test = train_test_split(lemmas_dict[lemmas_key], targets, random_state=RANDOM_STATE, test_size=test_size)
    tf_idf = TfidfVectorizer(stop_words=stopwords).fit(lemmas_train)
    X_train = tf_idf.transform(lemmas_train)
    X_test = tf_idf.transform(lemmas_test)
    feature_processing_describe = ', TfidfVectorizer'
    return X_train, X_test, y_train, y_test, feature_processing_describe

In [14]:
# добавим отдельно функцию которая при создании X_ добавляет длину текста в качестве еще одного обучающего признака 
def column_stack_sparse_matrix(X_sparse, array_1d_to_add):
    add_to_X = csr_matrix(array_1d_to_add).T
    return hstack((X_sparse, add_to_X))

def tfidf_features_length_split(lemmas_key, targets, test_size):
    lengths = lemmas_dict[lemmas_key].apply(len).values
    lemmas_train, lemmas_test, lengths_train, lengths_test, y_train, y_test = train_test_split(lemmas_dict[lemmas_key], lengths, targets, random_state=RANDOM_STATE, test_size=test_size)
    tf_idf = TfidfVectorizer(stop_words=stopwords).fit(lemmas_train)
    # теперь сформируем обучающие признаки, как tf_idf вектор + длина текста
    X_train = tf_idf.transform(lemmas_train)
    X_train = column_stack_sparse_matrix(X_train, lengths_train)
    X_test = tf_idf.transform(lemmas_test)
    X_test = column_stack_sparse_matrix(X_test, lengths_test)
    feature_processing_describe = ', TfidfVectorizer + text_length'
    return X_train, X_test, y_train, y_test, feature_processing_describe

## 2.1. Логистическая регрессия.

In [15]:
# попробуем логистическую регрессию на разных наборах лемм со след. гиперпараметрами:
test_size = 0.25
n_iter = 1000
class_w = 'balanced'
solver = 'liblinear'

In [16]:
%%time
# в цикле формируем сеты данных для лог. регрессии на основе различных наборов лемм, затем обучаем и тестируем модель
# TfidfVectorizer применяем только на train данных уже после разделения лемм на обучающие и тестировочные
for key in lemmas_dict:
    X_train, X_test, y_train, y_test, describe_str = tfidf_features_split(key, targets, test_size)
    fit_and_test(X_train, X_test, y_train, y_test, key, describe_str, LogisticRegression, max_iter=n_iter, class_weight=class_w, solver=solver, random_state=RANDOM_STATE)

CPU times: user 37.7 s, sys: 1.05 s, total: 38.7 s
Wall time: 35.2 s


In [17]:
all_results

,F1 on test,F1 on valid,F1 on train,Accuracy on test,time to fit,time to predict,time to vectorize,possible overfitting
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer",0.745,NaN,0.831,0.941,2.131,0.02,739.133,1.115
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer",0.751,NaN,0.834,0.942,2.836,0.019,57.207,1.111


In [18]:
# теперь посмотрим, изменится ли качество модели при добавлении длины текста
for key in lemmas_dict:
    X_train, X_test, y_train, y_test, describe_str = tfidf_features_length_split(key, targets, test_size)
    fit_and_test(X_train, X_test, y_train, y_test, key, describe_str, LogisticRegression, max_iter=n_iter, class_weight=class_w, solver=solver, random_state=RANDOM_STATE)

In [19]:
all_results

,F1 on test,F1 on valid,F1 on train,Accuracy on test,time to fit,time to predict,time to vectorize,possible overfitting
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer",0.745,NaN,0.831,0.941,2.131,0.02,739.133,1.115
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer",0.751,NaN,0.834,0.942,2.836,0.019,57.207,1.111
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer + text_length",0.746,NaN,0.831,0.941,2.862,0.172,739.133,1.114
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length",0.752,NaN,0.833,0.943,3.41,0.18,57.207,1.108


- Результат несколько неожиданный - самая простая (и быстрая) предобработка позволила получить более высокий результат F1 на тестовой выборке. 
- Добавление длины текста улучшило результат, но не сильно.
- Далее попробуем улучшить результаты логистической регрессии, увеличив кол-во данных для '1' класса или уменьшив кол-во данных для '0' класса

In [20]:
def split_classes(comments, targets):
    filter_zero = (targets == 0)
    filter_one = (targets == 1)
    comments_positive = comments[filter_zero]
    targets_positive = targets[filter_zero]
    comments_negative = comments[filter_one]
    targets_negative = targets[filter_one]
    return comments_positive, comments_negative, targets_positive, targets_negative

def upsampling(all_comments, targets, repeat):
    comments_p, comments_n, targets_p, targets_n = split_classes(all_comments, targets)
    comments_upsample = pd.concat([comments_p] + repeat * [comments_n])
    targets_upsample = pd.concat([targets_p] + repeat * [targets_n])
    return shuffle(comments_upsample, targets_upsample, random_state = RANDOM_STATE)

def downsampling(all_comments, targets, fraction):
    comments_p, comments_n, targets_p, targets_n = split_classes(all_comments, targets)
    comments_p = comments_p.sample(frac=fraction, random_state = RANDOM_STATE)
    targets_p = targets_p.sample(frac=fraction, random_state = RANDOM_STATE)
    comments_downsample = pd.concat([comments_p, comments_n])
    targets_downsample = pd.concat([targets_p, targets_n])
    return shuffle(comments_downsample, targets_downsample, random_state = RANDOM_STATE)

def tfidf_after_splitting_balancing(lemmas_key, targets, sampling_mult, test_size):
    lemmas_train, lemmas_test, y_train, y_test = train_test_split(lemmas_dict[lemmas_key], targets, random_state=RANDOM_STATE, test_size=test_size)
    if sampling_mult > 1:
        feature_processing_describe = f', upsampling x {sampling_mult}, TfidfVectorizer'
        lemmas_train_balanced, y_train_balanced = upsampling(lemmas_train, y_train, sampling_mult)
    else:
        feature_processing_describe = f', downsampling x {sampling_mult}, TfidfVectorizer'
        lemmas_train_balanced, y_train_balanced = downsampling(lemmas_train, y_train, sampling_mult)
    tf_idf = TfidfVectorizer(stop_words=stopwords).fit(lemmas_train_balanced)
    X_train_balanced = tf_idf.transform(lemmas_train_balanced)
    X_test = tf_idf.transform(lemmas_test)
    return X_train_balanced, X_test, y_train_balanced, y_test, feature_processing_describe

In [21]:
lemmas_key = 'clearing, splitting, lemmatization WITHOUT pos-tag'
for params in [(9, None), (4, 'balanced'), (0.5, 'balanced')]:
    X_train, X_test, y_train, y_test, describe_str = tfidf_after_splitting_balancing(lemmas_key, targets, params[0], test_size)
    fit_and_test(X_train, X_test, y_train, y_test, lemmas_key, describe_str, LogisticRegression, max_iter=n_iter, class_weight=params[1], solver=solver, random_state=RANDOM_STATE)

In [22]:
all_results

,F1 on test,F1 on valid,F1 on train,Accuracy on test,time to fit,time to predict,time to vectorize,possible overfitting
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer",0.745,NaN,0.831,0.941,2.131,0.02,739.133,1.115
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer",0.751,NaN,0.834,0.942,2.836,0.019,57.207,1.111
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer + text_length",0.746,NaN,0.831,0.941,2.862,0.172,739.133,1.114
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length",0.752,NaN,0.833,0.943,3.41,0.18,57.207,1.108
"LogisticRegression, max_iter=1000, class_weight=None, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, upsampling x 9, TfidfVectorizer",0.748,NaN,0.973,0.942,3.885,0.026,57.207,1.301
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, upsampling x 4, TfidfVectorizer",0.748,NaN,0.946,0.942,3.266,0.028,57.207,1.265
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, downsampling x 0.5, TfidfVectorizer",0.732,NaN,0.886,0.936,1,0.012,57.207,1.21


## 2.1. Выводы.
- Методом логистической регрессии формально удалось достигнуть требуемого качества по метрике F1. Лучший результат на тестовой выборке 0,752.
- Результат не сильно изменился при добавлении к tf_idf векторам еще и длины текста, тем не менее с таким дополнительным обучающим признаком модель продемонстрировала более высокий  F1 на тестовой выборке и меньшую степень переобученности.
- Upsampling/downsampling не улучшили результата F1, но увеличили показатель возможной переобученности.

## 2.2. LightGBM.

Для обучения модели LightGBM будем использовать лучший набор лемм, выявленный на этапе тестирования моделей логистической регресии, к обучающим признакам (векторизация tf-idf) будем добавлять длину текста. В отличие от процесса обучения модели логистической регрессии нам потребуется разбиение на обучающую, валидационную и тестовую выборки. Связано это с тем, что модель LightGBM  можно обучать одновременно с контролем logloss на валидационной выборке и не допустить переобученности, прекратив дальнейшие интерации (увеличение кол-ва деревьев), если logloss перестал уменьшаться.

In [23]:
# в связи с изменением методики обучения модифицируем нашу функцию обучения и измерения F1
def fit_valid_and_test(X_train, X_valid, X_test, y_train, y_valid, y_test, lemmas_key, feature_processing_describe, model_method, **kw_model):
    start_fit = time.time()
    model = model_method(**kw_model)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=5, verbose=100)
    end_fit = time.time()
    y_test_predict = model.predict(X_test)
    y_train_predict = model.predict(X_train)
    y_valid_predict = model.predict(X_valid)
    end_predict = time.time()
    model_name = model.__class__.__name__ + f', best iteration {model.best_iteration_}, ' + ', '.join([f'{i}={kw_model[i]}' for i in kw_model])
    index_row = model_name + '. Features preparing: ' + lemmas_key + feature_processing_describe
    f1_test = round(f1_score(y_test, y_test_predict), 3)
    f1_train = round(f1_score(y_train, y_train_predict), 3)
    all_results.loc[index_row, name_for['F1_test']] = f1_test
    all_results.loc[index_row, name_for['F1_valid']] = round(f1_score(y_valid, y_valid_predict), 3)
    all_results.loc[index_row, name_for['F1_train']] = f1_train
    all_results.loc[index_row, name_for['Accuracy']] = round(accuracy_score(y_test, y_test_predict), 3)
    all_results.loc[index_row, name_for['time_fit']] = round(end_fit - start_fit, 3)
    all_results.loc[index_row, name_for['time_predict']] = round(end_predict - end_fit, 3)
    all_results.loc[index_row, name_for['time_lemma']] = lemmas_time_dict[lemmas_key]
    all_results.loc[index_row, name_for['ratio']] = round(f1_train / f1_test,3)


# в связи с разбиением на три выборки также модифицируем функции разбиения на выборки и векторизации
def tfidf_features_split_with_valid(lemmas_key, targets, valid_size):
    lengths = lemmas_dict[lemmas_key].apply(len).values
    lemmas_train, lemmas_test, lengths_train, lengths_test, y_train, y_test = train_test_split(lemmas_dict[lemmas_key], lengths, targets, random_state=RANDOM_STATE, test_size=(1 - valid_size))
    lemmas_valid, lemmas_test, lengths_valid, lengths_test, y_valid, y_test = train_test_split(lemmas_test, lengths_test, y_test, random_state=RANDOM_STATE, test_size=0.5)
    tf_idf = TfidfVectorizer(stop_words=stopwords).fit(lemmas_train)
    X_train = tf_idf.transform(lemmas_train)
    X_train = column_stack_sparse_matrix(X_train, lengths_train)
    X_valid = tf_idf.transform(lemmas_valid)
    X_valid = column_stack_sparse_matrix(X_valid, lengths_valid)
    X_test = tf_idf.transform(lemmas_test)
    X_test = column_stack_sparse_matrix(X_test, lengths_test)
    feature_processing_describe = ', TfidfVectorizer + text_length, train+valid+test'
    return X_train, X_valid, X_test, y_train, y_valid, y_test, feature_processing_describe

In [24]:
# для выбора модели LightGBM будем использовать лучший набор лемм, 
# выявленный на этапе тестирования моделей логистической регресии
lemmas_key = 'clearing, splitting, lemmatization WITHOUT pos-tag'
valid_size = 0.6
X_train, X_valid, X_test, y_train, y_valid, y_test, describe_str = tfidf_features_split_with_valid(lemmas_key, targets, valid_size)

In [25]:
n_estimators_max = 1000
for max_leaves in [31, 70]:
    fit_valid_and_test(X_train, X_valid, X_test, y_train, y_valid, y_test, lemmas_key, describe_str, lgbm.LGBMClassifier, n_estimators=n_estimators_max, num_leaves=max_leaves, random_state=RANDOM_STATE)

Training until validation scores don't improve for 5 rounds
[100]	valid_0's binary_logloss: 0.127117
[200]	valid_0's binary_logloss: 0.119481
Early stopping, best iteration is:
[274]	valid_0's binary_logloss: 0.117806


/Users/sergeibovdei/opt/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:497: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


Training until validation scores don't improve for 5 rounds
[100]	valid_0's binary_logloss: 0.117844
Early stopping, best iteration is:
[158]	valid_0's binary_logloss: 0.116228


In [26]:
all_results

,F1 on test,F1 on valid,F1 on train,Accuracy on test,time to fit,time to predict,time to vectorize,possible overfitting
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer",0.745,NaN,0.831,0.941,2.131,0.02,739.133,1.115
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer",0.751,NaN,0.834,0.942,2.836,0.019,57.207,1.111
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer + text_length",0.746,NaN,0.831,0.941,2.862,0.172,739.133,1.114
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length",0.752,NaN,0.833,0.943,3.41,0.18,57.207,1.108
"LogisticRegression, max_iter=1000, class_weight=None, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, upsampling x 9, TfidfVectorizer",0.748,NaN,0.973,0.942,3.885,0.026,57.207,1.301
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, upsampling x 4, TfidfVectorizer",0.748,NaN,0.946,0.942,3.266,0.028,57.207,1.265
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, downsampling x 0.5, TfidfVectorizer",0.732,NaN,0.886,0.936,1,0.012,57.207,1.21
"LGBMClassifier, best iteration 274, n_estimators=1000, num_leaves=31, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length, train+valid+test",0.759,0.773,0.849,0.957,106.455,15.418,57.207,1.119
"LGBMClassifier, best iteration 158, n_estimators=1000, num_leaves=70, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length, train+valid+test",0.756,0.776,0.875,0.957,136.198,20.005,57.207,1.157


In [27]:
# увеличение максимального кол-ва листьев по сравнению с дефолтным практически не улучшает качество модели, 
# но увеличивает риск переобученности - зафиксируем лучшие параметры модели LGBMClassifier
# и переобучим модель на более общей выборке (разобьем весь сет на обучающий и тестовый)
n_estimators_best = 274
num_leaves_best = 31
X_train, X_test, y_train, y_test, describe_str = tfidf_features_length_split(key, targets, test_size)
# print('Lemmas splitted, features vectorized')
fit_and_test(X_train, X_test, y_train, y_test, lemmas_key, describe_str, lgbm.LGBMClassifier, n_estimators=n_estimators_best, num_leaves=num_leaves_best, random_state=RANDOM_STATE)
all_results

/Users/sergeibovdei/opt/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:497: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


,F1 on test,F1 on valid,F1 on train,Accuracy on test,time to fit,time to predict,time to vectorize,possible overfitting
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer",0.745,NaN,0.831,0.941,2.131,0.02,739.133,1.115
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer",0.751,NaN,0.834,0.942,2.836,0.019,57.207,1.111
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer + text_length",0.746,NaN,0.831,0.941,2.862,0.172,739.133,1.114
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length",0.752,NaN,0.833,0.943,3.41,0.18,57.207,1.108
"LogisticRegression, max_iter=1000, class_weight=None, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, upsampling x 9, TfidfVectorizer",0.748,NaN,0.973,0.942,3.885,0.026,57.207,1.301
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, upsampling x 4, TfidfVectorizer",0.748,NaN,0.946,0.942,3.266,0.028,57.207,1.265
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, downsampling x 0.5, TfidfVectorizer",0.732,NaN,0.886,0.936,1,0.012,57.207,1.21
"LGBMClassifier, best iteration 274, n_estimators=1000, num_leaves=31, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length, train+valid+test",0.759,0.773,0.849,0.957,106.455,15.418,57.207,1.119
"LGBMClassifier, best iteration 158, n_estimators=1000, num_leaves=70, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length, train+valid+test",0.756,0.776,0.875,0.957,136.198,20.005,57.207,1.157
"LGBMClassifier, n_estimators=274, num_leaves=31, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length",0.774,NaN,0.838,0.96,142.534,17.193,57.207,1.083


## 2.2. Выводы.
- Для обучения модели LightGBM использовался лучший набор лемм, выявленный на этапе тестирования моделей логистической регресии, к обучающим признакам (векторизация tf-idf) была добавлена длина текста. 
- Лучшая найденная модель LightGBM продемнострировала наиболее высокий результат по метрике F1 = 0.774 и мЕньшую степень возможной переобученности.

## 2.3. Обучение с использованием BERT

#### 2.3.1. Расчет имбедингов - можно их не рассчитывать, а загрузить из файла. Тогда следует сразу перейти к пункту 2.3.2.

In [173]:
# инициируем BERT tokenizer
path_to_bert = '/datasets/ds_bert/'
vocab_file = 'vocab.txt'
tokenizer_bert = transformers.BertTokenizer(vocab_file=(path_to_bert + vocab_file))

In [174]:
"""Мы будем использовать уже обученную BERT model, которая имеет ограничение на макс кол-во токенов в каждом тексте,
поэтому необходимо из имеющегося набора комментариев, как минимум, отобрать только те, длина которых 
не превышает 512 токенов. Мы сделаем более сильное ограничение на длину текста так, чтобы кол-во токенов примерно 
соответствовало кол-ву токенов в упражнении тренажера - таким образом мы, во-первых, ускорим работу BERT векторизатора
и, во-вторых, максимально приблизим параметры нашей задачи к задаче из тренажера, 
поскольку не все детали использованного там кода до конца понятны. """ 
text_size = df_comments['text'].apply(len)
max_text_size = 120
df_comments_text_short_only = df_comments[ text_size < max_text_size]

In [175]:
# проверим, насколько мы уменьшили первоначальную выборку и не сильно ли изменилось соотношение классов
print(f'Кол-во комментариев, в каждом из которых не более {max_text_size} символов:', len(df_comments_text_short_only))
print(f'Доля токсичных комментариев:       {df_comments_text_short_only.toxic.mean():6.2%}')

Кол-во комментариев, в каждом из которых не более 120 символов: 50157
Доля токсичных комментариев:       15.42%


In [176]:
# общее кол-во объектов сократилось более чем в 3 раза, доля токсичных комментариев увеличилась
# использование даже обученной BERT модели для embeddings требует много времени, поэтому ограничим кол-во объектов
limited_no = 15000
samples_limited = df_comments_text_short_only.sample(n=limited_no, random_state=RANDOM_STATE)
samples_limited = samples_limited.reset_index(drop=True)

In [177]:
# проверим, что оставленные объекты не сильно отличаются по балансу классов
print('Кол-во комментариев для BERT модели: ', len(samples_limited))
print(f'Доля токсичных комментариев в них:  {samples_limited.toxic.mean():6.2%}')

Кол-во комментариев для BERT модели:  15000
Доля токсичных комментариев в них:  15.28%


In [178]:
%%time
# токенизируем комментарии, оставленные для BERT модели,
tokenized = samples_limited['text'].apply(lambda x: tokenizer_bert.encode(x, add_special_tokens=True))

# дополним 0 все комментарии, менее максимальной длины,
max_length = max(len(tokens) for tokens in tokenized)
padded = np.array([i + [0]*(max_length - len(i)) for i in tokenized.values])

# и создадим маску внимания
attention_mask = np.where(padded != 0, 1, 0)

CPU times: user 4.22 s, sys: 42.6 ms, total: 4.27 s
Wall time: 4.34 s


In [179]:
# проверим, достаточно ли мы ограничили длину текста
max_length

88

In [180]:
# загрузим предобученную BERT модель для английского языка
config_file = 'bert_config.json'
model_file = 'pytorch_model.bin'

config_bert = transformers.BertConfig.from_json_file(path_to_bert + config_file)
model_bert = transformers.BertModel.from_pretrained(path_to_bert + model_file, config=config_bert)

In [181]:
# вычисление embeddings будем производить траншами, обрабатывая 100 комментариев за один транш
start_time = time.time()
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
    with torch.no_grad():
        batch_embeddings = model_bert(batch, attention_mask=attention_mask_batch)
    embeddings.append(batch_embeddings[0][:,0,:].numpy())
features = np.concatenate(embeddings)
end_time = time.time()

In [ ]:
# время вычисления имбедингов для 15К текстов (из почти 160К) на локальном компьютере составило примерно час
# сохраним полученные имбединги в файл, который можно будет использовать в дальнейшем, не тратя время на расчет имбедингов
np.save('./features_embeddings_120_15K_text.npy', features)
targets = samples_limited['toxic']
np.save('./targets_embeddings_120_15K_text.npy', targets)
lemmas_time_dict[dict_key] = round(end_time-start_time, 3)

####  2.3.2. Если не рассчитывать имбединги, а загрузить их из локального файла, тогда исполнение кода продолжить отсюда.

In [28]:
# сначала инициируем строковые переменные, необходимые для заполнения таблицы  all_relults
limited_no = 15000
max_text_size = 120 
name_for_BERT_vectorizer = 'Features preparing: '
dict_key = f'BERT tokenizing, {limited_no} texts {max_text_size} length'
describe_vectorizing = ', embeddings with BERT conversational, english, cased, 12-layer, 768-hidden, 12-heads, 110M parameters'

In [29]:
# время вычисления имбедингов для 15К текстов (из почти 160К) на локальном компьютере составило примерно час
# загрузим полученные ранее имбединги из файла
features = np.load('./features_embeddings_120_15K_text.npy')
targets = np.load('./targets_embeddings_120_15K_text.npy')
# если расчет имбедингов не производится, тогда внесем примерное значение в словарь, где храним время векторизации данных
lemmas_time_dict[dict_key] = lemmas_time_dict[lemmas_key] * 60

In [30]:
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=test_size, random_state=RANDOM_STATE)

In [31]:
# обучим линейную регрессию и запишем результаты в общую таблицу
fit_and_test(X_train, X_test, y_train, y_test, dict_key, describe_vectorizing, LogisticRegression, solver=solver, random_state=RANDOM_STATE)
all_results

,F1 on test,F1 on valid,F1 on train,Accuracy on test,time to fit,time to predict,time to vectorize,possible overfitting
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer",0.745,NaN,0.831,0.941,2.131,0.02,739.133,1.115
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer",0.751,NaN,0.834,0.942,2.836,0.019,57.207,1.111
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer + text_length",0.746,NaN,0.831,0.941,2.862,0.172,739.133,1.114
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length",0.752,NaN,0.833,0.943,3.41,0.18,57.207,1.108
"LogisticRegression, max_iter=1000, class_weight=None, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, upsampling x 9, TfidfVectorizer",0.748,NaN,0.973,0.942,3.885,0.026,57.207,1.301
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, upsampling x 4, TfidfVectorizer",0.748,NaN,0.946,0.942,3.266,0.028,57.207,1.265
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, downsampling x 0.5, TfidfVectorizer",0.732,NaN,0.886,0.936,1,0.012,57.207,1.21
"LGBMClassifier, best iteration 274, n_estimators=1000, num_leaves=31, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length, train+valid+test",0.759,0.773,0.849,0.957,106.455,15.418,57.207,1.119
"LGBMClassifier, best iteration 158, n_estimators=1000, num_leaves=70, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length, train+valid+test",0.756,0.776,0.875,0.957,136.198,20.005,57.207,1.157
"LGBMClassifier, n_estimators=274, num_leaves=31, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length",0.774,NaN,0.838,0.96,142.534,17.193,57.207,1.083


In [32]:
# найдем лучшую интерацию для LightGBM
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.4, random_state=RANDOM_STATE)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=RANDOM_STATE)
fit_valid_and_test(X_train, X_valid, X_test, y_train, y_valid, y_test, dict_key, describe_vectorizing, lgbm.LGBMClassifier, n_estimators=1000, num_leaves=num_leaves_best, random_state=RANDOM_STATE)

Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.226409


In [33]:
all_results

,F1 on test,F1 on valid,F1 on train,Accuracy on test,time to fit,time to predict,time to vectorize,possible overfitting
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer",0.745,NaN,0.831,0.941,2.131,0.02,739.133,1.115
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer",0.751,NaN,0.834,0.942,2.836,0.019,57.207,1.111
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer + text_length",0.746,NaN,0.831,0.941,2.862,0.172,739.133,1.114
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length",0.752,NaN,0.833,0.943,3.41,0.18,57.207,1.108
"LogisticRegression, max_iter=1000, class_weight=None, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, upsampling x 9, TfidfVectorizer",0.748,NaN,0.973,0.942,3.885,0.026,57.207,1.301
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, upsampling x 4, TfidfVectorizer",0.748,NaN,0.946,0.942,3.266,0.028,57.207,1.265
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, downsampling x 0.5, TfidfVectorizer",0.732,NaN,0.886,0.936,1,0.012,57.207,1.21
"LGBMClassifier, best iteration 274, n_estimators=1000, num_leaves=31, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length, train+valid+test",0.759,0.773,0.849,0.957,106.455,15.418,57.207,1.119
"LGBMClassifier, best iteration 158, n_estimators=1000, num_leaves=70, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length, train+valid+test",0.756,0.776,0.875,0.957,136.198,20.005,57.207,1.157
"LGBMClassifier, n_estimators=274, num_leaves=31, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length",0.774,NaN,0.838,0.96,142.534,17.193,57.207,1.083


In [34]:
best_iter_lgbm_bert = 71
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.25, random_state=RANDOM_STATE)
fit_and_test(X_train, X_test, y_train, y_test, dict_key, describe_vectorizing, lgbm.LGBMClassifier, n_estimators=best_iter_lgbm_bert, num_leaves=num_leaves_best, random_state=RANDOM_STATE)
all_results

,F1 on test,F1 on valid,F1 on train,Accuracy on test,time to fit,time to predict,time to vectorize,possible overfitting
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer",0.745,NaN,0.831,0.941,2.131,0.02,739.133,1.115
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer",0.751,NaN,0.834,0.942,2.836,0.019,57.207,1.111
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: no clearing, tokenization, lemmatization WITH pos-tag, TfidfVectorizer + text_length",0.746,NaN,0.831,0.941,2.862,0.172,739.133,1.114
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length",0.752,NaN,0.833,0.943,3.41,0.18,57.207,1.108
"LogisticRegression, max_iter=1000, class_weight=None, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, upsampling x 9, TfidfVectorizer",0.748,NaN,0.973,0.942,3.885,0.026,57.207,1.301
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, upsampling x 4, TfidfVectorizer",0.748,NaN,0.946,0.942,3.266,0.028,57.207,1.265
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, downsampling x 0.5, TfidfVectorizer",0.732,NaN,0.886,0.936,1,0.012,57.207,1.21
"LGBMClassifier, best iteration 274, n_estimators=1000, num_leaves=31, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length, train+valid+test",0.759,0.773,0.849,0.957,106.455,15.418,57.207,1.119
"LGBMClassifier, best iteration 158, n_estimators=1000, num_leaves=70, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length, train+valid+test",0.756,0.776,0.875,0.957,136.198,20.005,57.207,1.157
"LGBMClassifier, n_estimators=274, num_leaves=31, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length",0.774,NaN,0.838,0.96,142.534,17.193,57.207,1.083


## 2.3. Выводы.
- Использование BERT и корпуса текстов размером в 15000 (в 10 раз меньше, чем исходный корпус) и модели логистической регрессии позволило достигнуть самых высоких результатов по метрике F1 = 0.784.
- Использование BERT и корпуса текстов размером в 16000 (в 10 раз меньше, чем исходный корпус) не позволило добиться F1=0.75 для моделей LightGBM.
- Надо отметить, что BERT векторизация даже с использованием уже предобученной BERT модели и небольшого корпуса текстов требует значительных ресурсов для векторизации.

# 3. Выводы.

In [36]:
all_results[all_results['F1 on test'] > 0.75].sort_values(by='F1 on test', ascending=False)

,F1 on test,F1 on valid,F1 on train,Accuracy on test,time to fit,time to predict,time to vectorize,possible overfitting
"LogisticRegression, solver=liblinear, random_state=134. Features preparing: BERT tokenizing, 15000 texts 120 length, embeddings with BERT conversational, english, cased, 12-layer, 768-hidden, 12-heads, 110M parameters",0.784,NaN,0.842,0.939,5.115,0.078,3432.42,1.074
"LGBMClassifier, n_estimators=274, num_leaves=31, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length",0.774,NaN,0.838,0.96,142.534,17.193,57.207,1.083
"LGBMClassifier, best iteration 274, n_estimators=1000, num_leaves=31, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length, train+valid+test",0.759,0.773,0.849,0.957,106.455,15.418,57.207,1.119
"LGBMClassifier, best iteration 158, n_estimators=1000, num_leaves=70, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length, train+valid+test",0.756,0.776,0.875,0.957,136.198,20.005,57.207,1.157
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer + text_length",0.752,NaN,0.833,0.943,3.41,0.18,57.207,1.108
"LogisticRegression, max_iter=1000, class_weight=balanced, solver=liblinear, random_state=134. Features preparing: clearing, splitting, lemmatization WITHOUT pos-tag, TfidfVectorizer",0.751,NaN,0.834,0.942,2.836,0.019,57.207,1.111


- Наилучшего результата по метрике F1 удалось достигнуть при использовании BERT векторизации, даже сократив используемый для обучения корпус текстов в 10 раз. Метрика F1 на тестовой выборке составили 0.784.
- LightGBM при использовании TF-IDF векторизации позволил достичь самой лучшей accuracy и уровня F1 = 0.774. Время, затраченное на обучение и векторизацию/токенизацию/очистку текстов, значительно меньше, чем для BERT.
- Самая быстрая модель по времени предобработки текстов и обучения - модель логистической регрессии и TF-IDF векторизации - позволила получить F1 = 0.752.

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны